## Installing Libraries

In [1]:
!pip install nltk
!pip install transformers --quiet

import re
import nltk
import string
import os, gc
import pandas as pd
import tensorflow as tf
from transformers import TFAutoModel
from transformers import AutoTokenizer
nltk.download('stopwords')

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


D0503 11:20:33.752462009   10045 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0503 11:20:33.752489763   10045 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0503 11:20:33.752493295   10045 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0503 11:20:33.752496057   10045 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0503 11:20:33.752498517   10045 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0503 11:20:33.752501124   10045 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0503 11:20:33.752503664   10045 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0503 11:20:33.

True

## Setting data paths

In [2]:
## Corresponding to Original data ##
original_data_path = "../input/jigsaw-multilingual-toxic-comment-classification/"
train_file_1_path = original_data_path + "jigsaw-toxic-comment-train.csv"
train_file_2_path = original_data_path + "jigsaw-unintended-bias-train.csv"
submisison_file_path = original_data_path + "sample_submission.csv"
test_file_path = original_data_path + "test.csv"
validation_file_path = original_data_path + "validation.csv"

####### Columns to pick #######
train_file_1_cols = ['comment_text', "toxic"]
train_file_2_cols = ['comment_text', "toxic"]
validation_file_cols = ['comment_text', 'toxic']
test_file_cols = ["content"]

## Corresponding to Google API ##
google_train_data = "../input/jigsaw-train-multilingual-coments-google-api/"
spanish_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-es-cleaned.csv"
french_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-fr-cleaned.csv"
italian_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-it-cleaned.csv"
portugese_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-pt-cleaned.csv"
russian_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-ru-cleaned.csv"
turkish_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-tr-cleaned.csv"

####### Columns to pick #######
spanish_cols = ['comment_text', "toxic"]
french_cols = ['comment_text', "toxic"]
italian_cols = ['comment_text', "toxic"]
portugese_cols = ['comment_text', "toxic"]
russian_cols = ['comment_text', "toxic"]
turkish_cols = ['comment_text', "toxic"]

## Reading & using the translated data

In [3]:
## Reading csv files ##
train1 = pd.read_csv(train_file_1_path, usecols=train_file_1_cols)
valid = pd.read_csv(validation_file_path, usecols=validation_file_cols)
test = pd.read_csv(test_file_path, usecols=test_file_cols)
sub = pd.read_csv(submisison_file_path)

## Translated data ##
spanish_df = pd.read_csv(spanish_cleaned_data_path, usecols=spanish_cols)
french_df = pd.read_csv(french_cleaned_data_path, usecols=french_cols)
italian_df = pd.read_csv(italian_cleaned_data_path, usecols=italian_cols)
portugese_df = pd.read_csv(portugese_cleaned_data_path, usecols=portugese_cols)
russian_df = pd.read_csv(russian_cleaned_data_path, usecols=russian_cols)
turkish_df = pd.read_csv(turkish_cleaned_data_path, usecols=turkish_cols)

### Adding language column in each of the translated datasets ###
spanish_df['lang'] = 'es'
french_df['lang'] = 'fr'
italian_df['lang'] = 'it'
portugese_df['lang'] = 'pt'
russian_df['lang'] = 'ru'
turkish_df['lang'] = 'tr'
train1['lang'] = 'en'

## Combining the translated datasets ##
train = pd.concat([spanish_df, french_df, turkish_df, italian_df, russian_df, portugese_df, train1], axis=0)
del spanish_df, french_df, turkish_df, italian_df, russian_df, portugese_df, train1
gc.collect()

## Dropping missing datapoints with respect to comment_text ##
train = train.dropna(subset=['comment_text'])

## Preprocessing the data

In [4]:
## Removing URL's
def remove_urls(text):
    """
    Function takes a sentence and removes the URLs from the provided sentence.
    """
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# define a regular expression pattern to match non-alphanumeric characters
pattern = r"[^a-zA-Z0-9 ]"

# define a function to remove non-alphanumeric characters from a string using the pattern
def remove_non_alphanumeric(text):
    return re.sub(pattern, "", text)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text: str, punctuations: str):
    """
    Function takes a sentence & a punctuations string to remove punctuations from a sentence.
    """
    return text.translate(str.maketrans("", "", punctuations))

## Removing multi-characters
def remove_multiplechars(text):
    text = re.sub(r'(.)\1{3,}',r'\1', text)
    return text

def contraction_to_expansion(text: str, contractions_dict: dict):
    """
    Function takes a sentence (text) and a dictionary to map words like ain't to am not etc.
    and returns the final sentence.
    """
    if type(text) is str:
        for key in contractions_dict:
            value = contractions_dict[key]
            text = text.replace(key, value)
        return text
    else:
        return text
    
contractions = {"ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have",
                "'cause": "because","could've": "could have", "couldn't": "could not",
                "couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not",
                "hadn't": "had not", "hadn't've": "had not have","hasn't": "has not","haven't": "have not",
                "he'd": "he would","he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                "he's": "he is","how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is",
                "I'd": "I would", "I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am",
                "I've": "I have","isn't": "is not","it'd": "it had", "it'd've": "it would have","it'll": "it will",
                "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
                "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have",
                "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not", 
                "oughtn't've": "ought not have", "shan't": "shall not","sha'n't": "shall not",
                "shan't've": "shall not have","she'd": "she would","she'd've": "she would have", "she'll": "she will",
                "she'll've": "she will have","she's": "she is","should've": "should have","shouldn't": "should not",
                "shouldn't've": "should not have", "so've": "so have","so's": "so is","that'd": "that would",
                "that'd've": "that would have","that's": "that is","there'd": "there had",
                "there'd've": "there would have","there's": "there is","they'd": "they would",
                "they'd've": "they would have","they'll": "they will","they'll've": "they will have",
                "they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had",
                "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are",
                "we've": "we have","weren't": "were not","what'll": "what will", "what'll've": "what will have",
                "what're": "what are","what's": "what is","what've": "what have","when's": "when is",
                "when've": "when have", "where'd": "where did","where's": "where is","where've": "where have",
                "who'll": "who will","who'll've": "who will have","who's": "who is", "who've": "who have",
                "why's": "why is","why've": "why have","will've": "will have","won't": "will not",
                "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                "wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
                "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                "you'd": "you had", "you'd've": "you would have","you'll": "you you will",
                "you'll've": "you you will have","you're": "you are","you've": "you have"}

#################################################### CLEANING THE DATA ####################################################
def clean_text(text):
    text = text.replace("\n", "") ## Remove the next line character ##
    text = remove_urls(text) ## Remove the URLs ##
    text = re.compile(r'＼\(.+?\)/').sub("", text) ## Remove emoticons ##
    text = remove_non_alphanumeric(text) ## Remove Unicode characters ##
    text = remove_punctuation(text, PUNCT_TO_REMOVE) ## Removing Punctuations ##
    text = remove_multiplechars(text) ## Removing multiple characters in a word ##
    text = contraction_to_expansion(text, contractions)
    return text

stop_words = nltk.corpus.stopwords.words('english')

In [5]:
###### CLEANING THE DATA ######

train['comment_text'] = train['comment_text'].apply(lambda x: clean_text(x))
valid['comment_text'] = valid['comment_text'].apply(lambda x: clean_text(x))
test['content'] = test['content'].apply(lambda x: clean_text(x))

## TPU Configurations

In [6]:
#################### TPU Configurations ####################
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

AUTO = tf.data.experimental.AUTOTUNE
# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 240 ################################# CHANGE #############################
MODEL = 'xlm-roberta-large'
NUM_SAMPLES = 150000
RANDOM_STATE = 42
LEARNING_RATE = 1e-5 ######################### MAIN CHANGE ############################
WEIGHT_DECAY = 1e-6

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/

In [7]:
def encode(texts, tokenizer, max_len):
    """
    Function takes a list of texts, tokenizer (object)
    initialized from HuggingFace library, max_len (defines
    of how long the sentence lengths should be).
    """       
    tokens = tokenizer(texts, max_length=max_len, 
                    truncation=True, padding='max_length',
                    add_special_tokens=True, return_tensors='np')
    
    return tokens

## Encoding comment_text

In [8]:
## Intializing the tokenizer ##
tokenizer = AutoTokenizer.from_pretrained(MODEL)

train_inputs = encode(train['comment_text'].values.tolist(), 
                      tokenizer, max_len=MAX_LEN)
validation_inputs = encode(valid['comment_text'].values.tolist(),
                          tokenizer, max_len=MAX_LEN)
test_inputs = encode(test['content'].values.tolist(),
                    tokenizer, max_len=MAX_LEN)

## Preparing data using tf.data.Data API

In [9]:
def map_fn(input_ids, attention_mask, labels=None):
    if labels is not None:
        return {"input_ids": input_ids, "attention_mask": attention_mask}, labels
    else:
        return {"input_ids": input_ids, "attention_mask": attention_mask}

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs["input_ids"],
                                                    train_inputs["attention_mask"],
                                                   train['toxic']))
train_dataset = train_dataset.map(map_fn)
train_dataset = train_dataset.repeat().shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO)

validation_dataset = tf.data.Dataset.from_tensor_slices((validation_inputs['input_ids'],
                                                         validation_inputs['attention_mask'],
                                                        valid['toxic']))
validation_dataset = validation_dataset.map(map_fn)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(AUTO)

test_dataset = tf.data.Dataset.from_tensor_slices((test_inputs['input_ids'],
                                                  test_inputs['attention_mask']))
test_dataset = test_dataset.map(map_fn)
test_dataset = test_dataset.batch(BATCH_SIZE)

## Building the model

In [11]:
def build_model(transformer_layer, max_len):
    """
    Creating the model input layers, output layers,
    model definition and compilation.
        
    Returns: model object after compiling. 
    """
    input_ids = tf.keras.layers.Input(shape=(max_len,), 
                                      dtype=tf.int32, 
                                      name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), 
                                       dtype=tf.int32, 
                                       name="attention_mask")
    embeddings = transformer_layer(input_ids, 
                                 attention_mask=attention_mask)[1]
    x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
    y = tf.keras.layers.Dense(1, activation='sigmoid',name='outputs')(x)
    model = tf.keras.models.Model(inputs=[input_ids, attention_mask],
                             outputs=y)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,
                                         weight_decay=WEIGHT_DECAY)
    loss = tf.keras.losses.BinaryCrossentropy()
    AUC = tf.keras.metrics.AUC()
    
    model.compile(loss=loss, metrics=[AUC], optimizer=optimizer)    
    return model

## Loading model on TPUs

In [12]:
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer,
                        max_len=MAX_LEN)

All model checkpoint layers were used when initializing TFXLMRobertaModel.

All the layers of TFXLMRobertaModel were initialized from the model checkpoint at xlm-roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


## Training the model on Only English data for 1 epoch

In [13]:
train_steps_per_epoch = train_inputs['input_ids'].shape[0] // BATCH_SIZE
train_history = model.fit(train_dataset,
                         steps_per_epoch=train_steps_per_epoch,
                         validation_data=validation_dataset,
                         epochs=1) 

/usr/local/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:459: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  warnings.warn(
2023-05-03 11:31:43.128220: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_790/ReadVariableOp.
2023-05-03 11:31:46.038868: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_790/ReadVariableOp.


12216/12216 [==============================] - ETA: 0s - loss: 0.1289 - auc: 0.9597

2023-05-03 12:58:40.342502: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-05-03 12:58:40.875446: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


12216/12216 [==============================] - 5334s 421ms/step - loss: 0.1289 - auc: 0.9597 - val_loss: 0.2697 - val_auc: 0.9161


## Training the model on Validation data for 2 epochs further to fine-tune on it

In [15]:
validation_steps_per_epoch = validation_inputs['input_ids'].shape[0] // BATCH_SIZE
validation_history = model.fit(validation_dataset.repeat(),
                              steps_per_epoch=validation_steps_per_epoch,
                              epochs=2)

Epoch 1/2
62/62 [==============================] - 26s 419ms/step - loss: 0.2228 - auc: 0.9314
Epoch 2/2
62/62 [==============================] - 119s 417ms/step - loss: 0.1525 - auc: 0.9700


## Submit to Competition

In [16]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)

2023-05-03 13:02:04.212713: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-05-03 13:02:04.768241: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


499/499 [==============================] - 89s 136ms/step


- Public Leaderboard Score: 0.8861 and Private Leaderboard Score: 0.8866